## Cervix images exploration
### Based on an useful notebook from kaggle

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imread, imshow
#import cv2
%pylab

from subprocess import check_output
print(check_output(["ls", "/home/lcorona/Documents/kaggle/intel_competition/data/train"]).decode("utf8"))

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib
Type_1
Type_2
Type_3



/usr/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['imread', 'imshow']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
from glob import glob
basepath = '/home/lcorona/Documents/kaggle/intel_competition/data/train/'

all_cervix_images = []

for path in sorted(glob(basepath + "*")):
    cervix_type = path.split("/")[-1]
    cervix_images = sorted(glob(basepath + cervix_type + "/*"))
    all_cervix_images = all_cervix_images + cervix_images

all_cervix_images = pd.DataFrame({'imagepath': all_cervix_images})
all_cervix_images['filetype'] = all_cervix_images.apply(lambda row: row.imagepath.split(".")[-1], axis=1)
all_cervix_images['type'] = all_cervix_images.apply(lambda row: row.imagepath.split("/")[-2], axis=1)
all_cervix_images.head()

,imagepath,filetype,type
0,/home/lcorona/Documents/kaggle/intel_competiti...,jpg,Type_1
1,/home/lcorona/Documents/kaggle/intel_competiti...,jpg,Type_1
2,/home/lcorona/Documents/kaggle/intel_competiti...,jpg,Type_1
3,/home/lcorona/Documents/kaggle/intel_competiti...,jpg,Type_1
4,/home/lcorona/Documents/kaggle/intel_competiti...,jpg,Type_1


In [4]:
print('We have a total of {} images in the whole dataset'.format(all_cervix_images.shape[0]))
type_aggregation = all_cervix_images.groupby(['type', 'filetype']).agg('count')
type_aggregation_p = type_aggregation.apply(lambda row: 1.0*row['imagepath']/all_cervix_images.shape[0], axis=1)

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 8))

type_aggregation.plot.barh(ax=axes[0])
axes[0].set_xlabel("image count")
type_aggregation_p.plot.barh(ax=axes[1])
axes[1].set_xlabel("training size fraction")

We have a total of 1481 images in the whole dataset


In [5]:
fig = plt.figure(figsize=(12,8))

i = 1
for t in all_cervix_images['type'].unique():
    ax = fig.add_subplot(1,3,i)
    i+=1
    f = all_cervix_images[all_cervix_images['type'] == t]['imagepath'].values[0]
    plt.imshow(plt.imread(f))
    plt.title('sample for cervix {}'.format(t))

In [6]:
from collections import defaultdict

images = defaultdict(list)

for t in all_cervix_images['type'].unique():
    sample_counter = 0
    for _, row in all_cervix_images[all_cervix_images['type'] == t].iterrows():
        #print('reading image {}'.format(row.imagepath))
        try:
            img = imread(row.imagepath)
            sample_counter +=1
            images[t].append(img)
        except:
            print('image read failed for {}'.format(row.imagepath))
        if sample_counter > 100:
            break

image read failed for /home/lcorona/Documents/kaggle/intel_competition/data/train/Type_1/1339.jpg


In [7]:
dfs = []
for t in all_cervix_images['type'].unique():
    t_ = pd.DataFrame(
        {
            'nrows': list(map(lambda i: i.shape[0], images[t])),
            'ncols': list(map(lambda i: i.shape[1], images[t])),
            'nchans': list(map(lambda i: i.shape[2], images[t])),
            'type': t
        }
    )
    dfs.append(t_)

shapes_df = pd.concat(dfs, axis=0)
shapes_df_grouped = shapes_df.groupby(by=['nchans', 'ncols', 'nrows', 'type']).size().reset_index().sort_values(['type', 0], ascending=False)
shapes_df_grouped

,nchans,ncols,nrows,type,0
6,3,2448,3264,Type_3,70
9,3,3096,4128,Type_3,24
3,3,2322,4128,Type_3,3
13,3,3264,2448,Type_3,2
15,3,4128,3096,Type_3,2
8,3,3096,4128,Type_2,51
5,3,2448,3264,Type_2,44
12,3,3264,2448,Type_2,2
14,3,4128,3096,Type_2,2
2,3,2322,4128,Type_2,1
